# TD: Analyse en Composantes Principales (ACP)

Ce notebook a pour but de suivre les étapes du TD et d'appliquer les calculs de base de l'ACP en utilisant Python, notamment les bibliothèques `pandas` et `numpy`.

**Objectif :** Comprendre et appliquer pas à pas les calculs, en vérifiant les formules fondamentales .

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration pour l'affichage
sns.set_theme(style="whitegrid")
np.set_printoptions(precision=4, suppress=True)

## 2. Jeu de données

On commence par charger le jeu de données .

In [2]:
# Création du DataFrame pandas
data = {
    'X1 (Poids)': [60, 72, 57, 80],
    'X2 (Taille)': [165, 180, 170, 190],
    'X3 (Age)': [25, 30, 22, 28]
}
df = pd.DataFrame(data, index=['A', 'B', 'C', 'D'])

print("Jeu de données initial (X) :")
df

Jeu de données initial (X) :


,X1 (Poids),X2 (Taille),X3 (Age)
A,60,165,25
B,72,180,30
C,57,170,22
D,80,190,28


## 3. Étape préliminaire: Standardisation des données

Avant de commencer l'ACP, il faut centrer et réduire les variables .
La formule est : $z_{ij}=(x_{ij}-\overline{x}_{j})/s_{j}$ , où $s_j$ est l'écart-type *échantillon* (avec `n-1` au dénominateur, comme suggéré par la formule de covariance en ).

### 3.1. Calculez les moyennes et écarts-types

In [4]:
# 1. Calcul des moyennes
moyennes = df.mean()
print("Moyennes {x} :")
print(moyennes)

print("\n" + "-"*30 + "\n")

# 2. Calcul des écarts-types (ddof=1 pour l'écart-type échantillon, n-1)
ecarts_types = df.std(ddof=1)
print("Écarts-types (s_j) :")
print(ecarts_types)

Moyennes {x} :
X1 (Poids)      67.25
X2 (Taille)    176.25
X3 (Age)        26.25
dtype: float64

------------------------------

Écarts-types (s_j) :
X1 (Poids)     10.688779
X2 (Taille)    11.086779
X3 (Age)        3.500000
dtype: float64


> **Réponse :**
> 
> * **Moyennes :** [Recopiez vos moyennes ici]
> * **Écarts-types :** [Recopiez vos écarts-types ici]

### 3.2. Déduisez les valeurs centrées et réduites (Z)

In [5]:
# Calcul des données centrées-réduites (Z)
Z = (df - moyennes) / ecarts_types

print("Données centrées et réduites (Z) :")
Z

Données centrées et réduites (Z) :


,X1 (Poids),X2 (Taille),X3 (Age)
A,-0.678281,-1.014722,-0.357143
B,0.444391,0.338241,1.071429
C,-0.958950,-0.563735,-1.214286
D,1.192840,1.240216,0.500000


> **Réponse :**
> 
> [Recopiez votre matrice Z ici ou confirmez son calcul]

---

## 4. Calcul de la matrice de corrélation (R)

La matrice de corrélation R est la matrice de covariance des données centrées-réduites Z.

### 4.1. Construisez la matrice de corrélation R

In [12]:

R = df.corr()
print("Matrice de corrélation (R):")
print(R)

print("\n" + "-"*30 + "\n")

Matrice de corrélation (R):
             X1 (Poids)  X2 (Taille)  X3 (Age)
X1 (Poids)     1.000000     0.952850  0.826412
X2 (Taille)    0.952850     1.000000  0.676481
X3 (Age)       0.826412     0.676481  1.000000

------------------------------



### 4.2. Vérifiez que R est symétrique et que ses coefficients sont compris entre -1 et 1

In [13]:
# Vérification de la symétrie
est_symetrique = np.allclose(R.values, R.values.T)
print(f"La matrice R est-elle symétrique ? {est_symetrique}")

# Vérification des coefficients
min_val = R.values.min()
max_val = R.values.max()
coeffs_valides = (min_val >= -1) and (max_val <= 1)
print(f"Les coefficients sont-ils tous entre -1 et 1 ? {coeffs_valides}")

La matrice R est-elle symétrique ? True
Les coefficients sont-ils tous entre -1 et 1 ? True


---

## 5. Calcul des valeurs propres et vecteurs propres

On cherche les valeurs propres $(\lambda)$ et les vecteurs propres $(u)$ de la matrice R.
On utilise `numpy.linalg.eig` pour cela.

### 5.1, 5.2, 5.3. Calculer et Normaliser les Valeurs/Vecteurs Propres

In [ ]:
# Calcul des valeurs propres et vecteurs propres
valeurs_propres, vecteurs_propres = np.linalg.eig(R)

# Trier les valeurs propres et les vecteurs propres par ordre décroissant
idx = valeurs_propres.argsort()[::-1]   
lambda_ = valeurs_propres[idx]
u = vecteurs_propres[:,idx]

# Note : np.linalg.eig renvoie déjà des vecteurs propres normalisés (||u_k|| = 1)

print("Valeurs propres (\lambda_k) - (triées) :")
print(lambda_)

print("\n" + "-"*30 + "\n")

print("Vecteurs propres (u_k) - (triés et normalisés) :")
# On met ça dans un DataFrame pour la lisibilité
df_u = pd.DataFrame(u, index=R.index, columns=['u1', 'u2', 'u3'])
print(df_u)

### 5.4. Calculez la part de variance expliquée

In [ ]:
# La somme des valeurs propres doit être égale à la trace de R (nombre de variables)
print(f"Somme des valeurs propres : {np.sum(lambda_)}")
print(f"Nombre de variables : {len(R.columns)}")

# Part de variance expliquée
variance_expliquee = lambda_ / np.sum(lambda_)
variance_cumulee = np.cumsum(variance_expliquee)

df_variance = pd.DataFrame({
    'Valeur Propre': lambda_,
    'Variance Expliquée (%)': variance_expliquee * 100,
    'Variance Cumulée (%)': variance_cumulee * 100
}, index=['Axe 1', 'Axe 2', 'Axe 3'])

print("\n" + "-"*30 + "\n")
print("Variance expliquée par chaque axe :")
print(df_variance.round(2))

> **Réponse :**
> 
> * **Valeurs propres :** [\lambda_1 = ..., \lambda_2 = ..., \lambda_3 = ...]
> * **Vecteurs propres (u1, u2, u3) :** [Recopiez les vecteurs ici]
> * **Part de variance :** [Axe 1 explique ...%, Axe 2 explique ...%]

---

## 6. Calcul des composantes principales

On calcule les coordonnées des individus sur les nouveaux axes (les composantes principales) .
La formule est $F = ZU$ .

### 6.1. Calculez les coordonnées des individus sur les deux premiers axes

In [ ]:
# Z est notre DataFrame de données centrées-réduites
# u est notre matrice de vecteurs propres

# Conversion de Z en array numpy pour la multiplication matricielle
Z_array = Z.values

# Calcul des composantes principales (F = Z @ u)
F = Z_array @ u

# Stockage dans un DataFrame pour la lisibilité
df_F = pd.DataFrame(F, columns=['F1', 'F2', 'F3'], index=df.index)

print("Coordonnées des individus sur les axes principaux (F) :")
print(df_F.round(4))

print("\n" + "-"*30 + "\n")
print("Coordonnées pour le premier plan factoriel (F1, F2) :")
print(df_F[['F1', 'F2']].round(4))

### 6.2. Représentez les individus sur le plan factoriel (F1, F2)

In [ ]:
plt.figure(figsize=(10, 7))

# Nuage de points des individus
sns.scatterplot(data=df_F, x='F1', y='F2', s=100) 

# Ajout des labels pour chaque individu
for i in range(df_F.shape[0]):
    plt.text(x=df_F.F1[i] + 0.05, # léger décalage
             y=df_F.F2[i] + 0.05, 
             s=df_F.index[i],
             fontdict=dict(color='black', size=12))

# Ajout des axes (0,0)
plt.axhline(0, color='grey', linestyle='--')
plt.axvline(0, color='grey', linestyle='--')

# Labels et titre
var_f1 = df_variance.loc['Axe 1', 'Variance Expliquée (%)']
var_f2 = df_variance.loc['Axe 2', 'Variance Expliquée (%)']
plt.xlabel(f"F1 ({var_f1:.2f}%)")
plt.ylabel(f"F2 ({var_f2:.2f}%)")
plt.title("Plan factoriel des individus (F1, F2)")
plt.grid(True)
plt.show()

> **Réponse :**
> 
> [Interprétez le graphique : quels individus sont proches ? Quels sont opposés ? Que remarquez-vous ?]

---

## 7. Qualité de représentation des variables (cos²)

On mesure la corrélation entre les variables d'origine (X) et les nouveaux axes (F) .

### 7.1. Calculez les corrélations entre chaque variable et les axes F1 et F2

In [ ]:
# Formule : corr(X_j, F_k) = sqrt(lambda_k) * u_jk 

sqrt_lambda = np.sqrt(lambda_)

# Multiplication de chaque vecteur propre (u) par le sqrt(lambda) correspondant
# u est (3, 3), sqrt_lambda est (3,)
corr_XF = u * sqrt_lambda

df_corr_XF = pd.DataFrame(corr_XF, 
                        index=df.columns, 
                        columns=['F1', 'F2', 'F3'])

print("Corrélations Variables/Axes (Cercle de corrélation) :")
print(df_corr_XF.round(4))

print("\n" + "-"*30 + "\n")
print("Corrélations pour le premier plan (F1, F2) :")
print(df_corr_XF[['F1', 'F2']].round(4))

### 7.2. En déduisez la qualité de représentation (cos²)

In [ ]:
# Formule : cos²(X_j, F_k) = (corr(X_j, F_k))^2 = lambda_k * u_jk^2 

cos2_XF = df_corr_XF**2

print("Qualité de représentation (cos²) des variables sur les axes :")
print(cos2_XF.round(4))

### 7.3. Interprétez: quelles variables sont bien représentées sur le premier plan ?

In [ ]:
# On peut calculer le cos² cumulé sur F1 et F2
cos2_plan12 = cos2_XF[['F1', 'F2']].sum(axis=1)
print("Qualité de représentation sur le plan F1+F2 :")
print(cos2_plan12.round(4))

> **Réponse :**
> 
> * **Variables bien représentées par F1 :** [Regardez les cos² élevés pour F1]
> * **Variables bien représentées par F2 :** [Regardez les cos² élevés pour F2]
> * **Variables bien représentées sur le plan (F1, F2) :** [Regardez la somme des cos² pour F1 et F2. Celles proches de 1 sont bien représentées.]

---

## 8. Contributions des variables aux axes

On regarde quelles variables "construisent" le plus chaque axe .

### 8.1. Calculez les contributions des variables

In [ ]:
# Formule : Ctr(X_j, F_k) = u_jk^2 
# La somme des contributions pour un axe k (sur toutes les variables j) doit valoir 1.
# Ctr(X_j, F_k) en % = u_jk^2 * 100 

# df_u est notre DataFrame de vecteurs propres (u)
contributions = df_u**2

# Vérifions que la somme par colonne (par axe) vaut 1
print("Vérification (Somme des contributions par axe) :")
print(contributions.sum(axis=0))

print("\n" + "-"*30 + "\n")

print("Contributions des variables aux axes (en %) :")
print((contributions * 100).round(2))

### 8.2. Interprétez: quelles variables déterminent le plus chaque axe?

> **Réponse :**
> 
> * **Variables contribuant le plus à F1 :** [Regardez les % les plus élevés dans la colonne F1]
> * **Variables contribuant le plus à F2 :** [Regardez les % les plus élevés dans la colonne F2]

---

## 9. Interprétation finale

Rassemblez toutes les informations pour interpréter l'ACP.

### 9.1. Quelle proportion de la variance totale est expliquée par les deux premiers axes ?

In [ ]:
print(df_variance.round(2))
print(f"\nVariance cumulée par F1+F2 : {df_variance.loc['Axe 2', 'Variance Cumulée (%)']:.2f} %" )

> **Réponse :**
> 
> [La proportion de variance expliquée par les deux premiers axes est de ... % (voir Section 5.4)]

### 9.2. Que représente le premier axe? (un facteur global, une opposition, etc.)

> **Réponse :**
> 
> [Pour interpréter F1, regardez :
> 1.  **Corrélations (Section 7.1) :** Quelles variables sont fortement corrélées (proches de +1 ou -1) à F1 ?
> 2.  **Contributions (Section 8.1) :** Quelles variables contribuent le plus à sa création ?
> 3.  **Individus (Section 6.2) :** Comment F1 sépare-t-il les individus ?
> 
> **Exemple d'interprétation :** "L'axe F1 est fortement et positivement corrélé aux variables X1, X2 et X3 (voir 7.1). Il représente donc un 'facteur de ...' (par exemple 'gabarit' ou 'croissance'). Les individus avec un F1 élevé (comme B et D) ont des valeurs élevées sur ces 3 variables, tandis que les individus avec un F1 bas (comme A et C) ont des valeurs plus faibles."]

### 9.3. Quelle variable semble indépendante ou spécifique ?

> **Réponse :**
> 
> [Une variable indépendante ou spécifique serait :
> * Faiblement corrélée aux autres (voir Matrice R, Section 4.1).
> * Mal représentée sur le premier plan (F1, F2) (voir cos² total, Section 7.3).
> * Fortement corrélée à un axe qui explique peu de variance (ex: F3).
> 
> Analysez vos résultats pour X1, X2 et X3.]

### 9.4. Quelle réduction de dimension est raisonnable ici ?

> **Réponse :**
> 
> [En se basant sur la variance cumulée (Section 5.4), on voit que les ... premiers axes expliquent ...% de l'information. (Règle de Kaiser : garder les axes avec $\lambda_k > 1$. Coude : regarder la chute des valeurs propres). Ici, une réduction à ... dimensions semble raisonnable.]

---

## Vérification avec Scikit-Learn

Vérifions rapidement nos calculs manuels avec la bibliothèque standard `sklearn`.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# 1. Standardisation
Z_sklearn = StandardScaler().fit_transform(df)
print("Z (sklearn) :\n", Z_sklearn)
print("\nZ (manuel) :\n", Z.values)
print("\nSont-ils identiques ?", np.allclose(Z_sklearn, Z.values))

print("\n" + "="*30 + "\n")

# 2. Lancer l'ACP
pca = PCA()
F_sklearn = pca.fit_transform(Z_sklearn)

print("Composantes (F) (sklearn) :\n", F_sklearn)
print("\nComposantes (F) (manuel) :\n", df_F.values)
print("\nNote : Les signes peuvent être inversés (ex: F1 manuel = -F1 sklearn). C'est normal, \n",
      "un vecteur propre u et son opposé -u sont mathématiquement équivalents.\n",
      "Ici, nos F1 et F2 manuels sont l'opposé de ceux de sklearn.")

print("\n" + "="*30 + "\n")

# 3. Vérifier la variance expliquée
print("Variance expliquée (sklearn) :\n", pca.explained_variance_ratio_)
print("\nVariance expliquée (manuel) :\n", variance_expliquee)
print("\nSont-ils identiques ?", np.allclose(pca.explained_variance_ratio_, variance_expliquee))

print("\n" + "="*30 + "\n")

# 4. Vérifier les vecteurs propres (appelés 'components_' dans sklearn)
print("Vecteurs propres (sklearn.components_) :\n", pca.components_)
print("\nVecteurs propres (manuel, u.T) :\n", u.T)
print("\nNote : Mêmes signes inversés que pour F, ce qui est cohérent.")